In [3]:
!pip install --upgrade tensorflow tensorflow-datasets


In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Set seed for reproducibility
tf.random.set_seed(42)


# Enable memory growth for GPUs (avoid crashes)
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

# Load PatchCamelyon dataset efficiently
(ds_train, ds_validation, ds_test), ds_info = tfds.load(
    'patch_camelyon',
    split=['train[:80%]', 'train[80%:]', 'validation'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# Verify loaded dataset
print(f"Dataset Info:\n{ds_info}")

# Example: view a sample image
import matplotlib.pyplot as plt

def show_sample(image, label):
    plt.imshow(image.numpy())
    plt.title(f"Label: {label.numpy()}")
    plt.axis('off')

for image, label in ds_train.take(1):
    show_sample(image, label)


In [5]:
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds

# Load the dataset
ds_train, ds_info = tfds.load('patch_camelyon', split='train', with_info=True, as_supervised=True)

# Display 5 sample images with labels
plt.figure(figsize=(15, 3))
for i, (image, label) in enumerate(ds_train.take(5)):
    plt.subplot(1, 5, i + 1)
    plt.imshow(image.numpy())
    plt.title(f'Label: {label.numpy()}')
    plt.axis('off')

plt.show()


In [6]:
# Import necessary libraries
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

# Enable GPU memory growth to avoid runtime crashes
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

# Load PCam dataset with 80% train and 20% test split
(ds_train, ds_test), ds_info = tfds.load(
    'patch_camelyon',
    split=['train[:80%]', 'train[80%:]'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

# Verify dataset info
print(ds_info)

# Define preprocessing and augmentation functions
def preprocess(image, label):
    # Normalise images to [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)
    # Resize images to ensure consistent input size (if required by your models)
    image = tf.image.resize(image, [96, 96])
    return image, label

# Augmentation function according to PDM plan (rotation, flip, contrast)

def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.8, upper=1.2)
    image = tf.image.rot90(image, k=tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))
    image = tf.clip_by_value(image, 0.0, 1.0)  # Ensure pixel values are within [0,1]
    return image, label

# Apply preprocessing, augmentation, batching, caching, and prefetching
batch_size = 64

# Training dataset pipeline
ds_train = (
    ds_train
    .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    .map(augment, num_parallel_calls=tf.data.AUTOTUNE)
    .cache()
    .shuffle(1000)
    .batch(batch_size)
    .prefetch(tf.data.AUTOTUNE)
)

# Test dataset pipeline (no augmentation, just preprocessing)
ds_test = (
    ds_test
    .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(batch_size)
    .cache()
    .prefetch(tf.data.AUTOTUNE)
)

# Check a batch of training images
for images, labels in ds_train.take(1):
    plt.figure(figsize=(15, 5))
    for i in range(5):
        plt.subplot(1, 5, i+1)
        plt.imshow(images[i].numpy())
        plt.title(f'Label: {labels[i].numpy()}')
        plt.axis('off')
    plt.show()


In [10]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

# Load DenseNet121 model with pretrained weights (imagenet) and custom input shape
base_model = DenseNet121(input_shape=(96, 96, 3), include_top=False, weights='imagenet')

# Freeze base model layers to leverage pretrained features initially
base_model.trainable = False

# Define the complete model
model_densenet = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model
model_densenet.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

# Model summary
model_densenet.summary()


In [11]:
import tensorflow as tf
import numpy as np
import random
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(
    monitor='val_loss',      # Monitor validation loss to detect overfitting
    patience=5,              # Stop if no improvement for 5 consecutive epochs
    restore_best_weights=True,  # Restore the best model after early stopping
    verbose=1
)

# Model checkpoint to save the best model during training
model_checkpoint = ModelCheckpoint(
    filepath='best_densenet_model.h5',   # Save the model with the best validation performance
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

# Train the DenseNet-121 model for 20 epochs with callbacks
history_densenet = model_densenet.fit(
    ds_train,
    epochs=20,
    validation_data=ds_test,
    callbacks=[early_stopping, model_checkpoint]  # Adding callbacks
)

# Save the final model explicitly (if not already saved by ModelCheckpoint)
model_densenet.save('final_densenet_model.h5')
print("Training complete. Model saved.")


In [12]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Function to plot training curves with spaced x-axis
def plot_training_curves(history):
    epochs = np.arange(1, len(history.history['accuracy']) + 1)

    plt.figure(figsize=(18, 5))

    # Accuracy Plot
    plt.subplot(1, 3, 1)
    plt.plot(epochs, history.history['accuracy'], label='Training Accuracy')
    plt.plot(epochs, history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.xticks(np.arange(1, len(epochs) + 1, step=2))  # Spaced x-axis
    plt.legend()

    # Loss Plot
    plt.subplot(1, 3, 2)
    plt.plot(epochs, history.history['loss'], label='Training Loss')
    plt.plot(epochs, history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.xticks(np.arange(1, len(epochs) + 1, step=2))  # Spaced x-axis
    plt.legend()

    # AUC-ROC Plot
    plt.subplot(1, 3, 3)
    plt.plot(epochs, history.history['auc'], label='Training AUC')
    plt.plot(epochs, history.history['val_auc'], label='Validation AUC')
    plt.title('AUC-ROC')
    plt.xlabel('Epoch')
    plt.ylabel('AUC')
    plt.xticks(np.arange(1, len(epochs) + 1, step=2))  # Spaced x-axis
    plt.legend()

    plt.tight_layout()
    plt.show()

# Function to evaluate the model
def evaluate_model(model, dataset):
    y_true, y_pred = [], []
    for images, labels in dataset:
        predictions = model.predict(images)
        predictions = np.round(predictions).astype(int).flatten()
        y_true.extend(labels.numpy())
        y_pred.extend(predictions)

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)

    print("\nDenseNet-121 Model Evaluation Metrics:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"AUC-ROC: {auc:.4f}")

# Plot training curves and evaluate the model after 20 epochs
plot_training_curves(history_densenet)
evaluate_model(model_densenet, ds_test)


In [13]:
import tensorflow as tf
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import random

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Load DenseNet121 model with pretrained weights (ImageNet) and custom input shape
base_model = DenseNet121(input_shape=(96, 96, 3), include_top=False, weights='imagenet')

# Unfreeze all layers in the base model for fine-tuning
base_model.trainable = True

# Define the complete model
model_densenet = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model with a low learning rate for fine-tuning
model_densenet.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

# Early stopping to prevent overfitting and model checkpointing
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
model_checkpoint = ModelCheckpoint(
    'best_densenet_finetune.h5',
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

# Fine-tune the model for additional 10 epochs
history_finetune = model_densenet.fit(
    ds_train,
    epochs=10,
    validation_data=ds_test,
    callbacks=[early_stopping, model_checkpoint]
)

# Save the final model after fine-tuning
model_densenet.save('densenet_finetuned.h5')
print("Model saved after fine-tuning!")


In [14]:
# Required imports for visualisation and evaluation
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Plot training curves after fine-tuning
def plot_training_curves(history):
    plt.figure(figsize=(18, 5))

    plt.subplot(1, 3, 1)
    plt.plot(history.history['accuracy'], label='Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 3, 2)
    plt.plot(history.history['loss'], label='Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 3, 3)
    plt.plot(history.history['auc'], label='AUC')
    plt.plot(history.history['val_auc'], label='Validation AUC')
    plt.title('AUC-ROC')
    plt.xlabel('Epoch')
    plt.ylabel('AUC')
    plt.legend()

    plt.show()

# Evaluate model performance on test data
def evaluate_model(model, dataset):
    y_true, y_pred = [], []
    for images, labels in dataset:
        predictions = model.predict(images)
        predictions = np.round(predictions).astype(int)
        y_true.extend(labels.numpy())
        y_pred.extend(predictions.flatten())

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)

    print("\nModel Evaluation Metrics:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"AUC-ROC: {auc:.4f}")

# Run these functions after fine-tuning
plot_training_curves(history_finetune)
evaluate_model(model_densenet, ds_test)


In [15]:
# Required imports for visualisation and evaluation
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Plot training curves for each phase separately
def plot_training_curves(history, title_prefix=""):
    epochs = np.arange(1, len(history.history['accuracy']) + 1)

    plt.figure(figsize=(18, 5))

    # Accuracy Plot
    plt.subplot(1, 3, 1)
    plt.plot(epochs, history.history['accuracy'], label='Training Accuracy', marker='o')
    plt.plot(epochs, history.history['val_accuracy'], label='Validation Accuracy', marker='x')
    plt.title(f'{title_prefix} Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.xticks(np.arange(1, len(epochs) + 1, step=1))
    plt.ylim(0.75, 0.95)
    plt.legend()

    # Loss Plot
    plt.subplot(1, 3, 2)
    plt.plot(epochs, history.history['loss'], label='Training Loss', marker='o')
    plt.plot(epochs, history.history['val_loss'], label='Validation Loss', marker='x')
    plt.title(f'{title_prefix} Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.xticks(np.arange(1, len(epochs) + 1, step=1))
    plt.ylim(0.2, 0.6)
    plt.legend()

    # AUC-ROC Plot
    plt.subplot(1, 3, 3)
    plt.plot(epochs, history.history['auc'], label='Training AUC', marker='o')
    plt.plot(epochs, history.history['val_auc'], label='Validation AUC', marker='x')
    plt.title(f'{title_prefix} AUC-ROC')
    plt.xlabel('Epoch')
    plt.ylabel('AUC')
    plt.xticks(np.arange(1, len(epochs) + 1, step=1))
    plt.ylim(0.85, 1.0)
    plt.legend()

    plt.tight_layout()
    plt.show()

# Evaluate model performance on test data
def evaluate_model(model, dataset):
    y_true, y_pred = [], []
    for images, labels in dataset:
        predictions = model.predict(images)
        predictions = np.round(predictions).astype(int)
        y_true.extend(labels.numpy())
        y_pred.extend(predictions.flatten())

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)

    print("\nModel Evaluation Metrics:")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"AUC-ROC: {auc:.4f}")

    return y_true, y_pred

# Plot confusion matrix for evaluation
def plot_confusion_matrix(y_true, y_pred, title="Confusion Matrix"):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(title)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

# Plot training curves separately for initial and fine-tuning phases
if 'history_densenet' in globals():
    plot_training_curves(history_densenet, title_prefix="Initial Training (20 Epochs)")

if 'history_finetune' in globals():
    plot_training_curves(history_finetune, title_prefix="Fine-Tuning (10 Epochs)")

# Run evaluation
y_true, y_pred = evaluate_model(model_densenet, ds_test)

# Plot confusion matrix
plot_confusion_matrix(y_true, y_pred, title="Confusion Matrix after Fine-Tuning")


In [16]:
# Required imports for advanced evaluation plots
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc, precision_recall_curve, f1_score, accuracy_score

def plot_roc_curve(y_true, y_pred_prob):
    fpr, tpr, _ = roc_curve(y_true, y_pred_prob)
    roc_auc = auc(fpr, tpr)

    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, color='blue', label=f'ROC Curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--', label='Random Guess')
    plt.xlabel('False Positive Rate (1 - Specificity)')
    plt.ylabel('True Positive Rate (Sensitivity)')
    plt.title('ROC Curve')
    plt.legend(loc='lower right')
    plt.show()

def plot_precision_recall_curve(y_true, y_pred_prob):
    precision, recall, _ = precision_recall_curve(y_true, y_pred_prob)
    f1 = f1_score(y_true, np.round(y_pred_prob))

    plt.figure(figsize=(8, 6))
    plt.plot(recall, precision, color='purple', label=f'PR Curve (F1 = {f1:.2f})')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc='upper right')
    plt.show()

def plot_f1_vs_threshold(y_true, y_pred_prob):
    thresholds = np.linspace(0, 1, 100)
    f1_scores = [f1_score(y_true, y_pred_prob >= t) for t in thresholds]

    plt.figure(figsize=(8, 6))
    plt.plot(thresholds, f1_scores, marker='o', color='green')
    plt.xlabel('Threshold')
    plt.ylabel('F1 Score')
    plt.title('F1 Score vs. Decision Threshold')
    plt.grid(True)
    plt.show()

def plot_accuracy_vs_threshold(y_true, y_pred_prob):
    thresholds = np.linspace(0, 1, 100)
    accuracies = [accuracy_score(y_true, y_pred_prob >= t) for t in thresholds]

    plt.figure(figsize=(8, 6))
    plt.plot(thresholds, accuracies, marker='x', color='red')
    plt.xlabel('Threshold')
    plt.ylabel('Accuracy')
    plt.title('Accuracy vs. Decision Threshold')
    plt.grid(True)
    plt.show()

# Evaluation function to get true labels and predicted probabilities
def get_predictions(model, dataset):
    y_true, y_pred_prob = [], []
    for images, labels in dataset:
        preds = model.predict(images).flatten()
        y_true.extend(labels.numpy())
        y_pred_prob.extend(preds)
    return np.array(y_true), np.array(y_pred_prob)

# Get predictions from the model
y_true, y_pred_prob = get_predictions(model_densenet, ds_test)

# Plot the advanced evaluation curves
plot_roc_curve(y_true, y_pred_prob)
plot_precision_recall_curve(y_true, y_pred_prob)
plot_f1_vs_threshold(y_true, y_pred_prob)
plot_accuracy_vs_threshold(y_true, y_pred_prob)


In [17]:
import matplotlib.pyplot as plt
import numpy as np

# Function to visualise predictions on a subset of images
def visualize_predictions(model, dataset, num_images=20):
    plt.figure(figsize=(20, 10))
    images_shown = 0

    for images, labels in dataset:
        predictions = model.predict(images)
        preds_rounded = np.round(predictions).astype(int).flatten()
        preds_probs = predictions.flatten()

        for i in range(images.shape[0]):
            plt.subplot(4, 5, images_shown + 1)
            plt.imshow(images[i].numpy())
            true_label = labels.numpy()[i]
            pred_label = preds_rounded[i]
            pred_prob = preds_probs[i]

            # Title includes true label, predicted label, and prediction confidence
            plt.title(f"True: {true_label} | Pred: {pred_label}\nProb: {pred_prob:.2%}",
                      color="green" if true_label == pred_label else "red")
            plt.axis('off')
            images_shown += 1

            if images_shown >= num_images:
                plt.tight_layout()
                plt.show()
                return

# Visualise model predictions on 20 test images
visualize_predictions(model_densenet, ds_test)
